<a href="https://colab.research.google.com/github/priyanshu7466/Python/blob/main/Deep_Learning/Distributed_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

In [2]:
(X_train, y_train), (X_test, y_test) = keras.datasets.cifar10.load_data()

170498071/170498071 [==============================] - 2s 0us/step


In [3]:
len(X_train)

50000

In [5]:
tf.test.is_built_with_cuda()

True

In [6]:
tf.__version__

'2.15.0'

In [7]:
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [8]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="4"

In [9]:
X_train.shape

(50000, 32, 32, 3)

In [10]:
y_train.shape

(50000, 1)

In [11]:
classes = ["airplane","automobile","bird","cat","deer","dog","frog","horse","ship","truck"]

In [12]:
classes[y_train[3][0]]

'deer'

In [13]:
X_train_scaled = X_train/255
X_test_scaled = X_test/255

In [14]:
y_train_categorical = keras.utils.to_categorical(
    y_train, num_classes=10, dtype='float32'
)
y_test_categorical = keras.utils.to_categorical(
    y_test, num_classes=10, dtype='float32'
)

In [22]:
def get_model():
  model = keras.Sequential(
      [keras.layers.Flatten(input_shape=(32,32,3)),
      keras.layers.Dense(3000, activation='relu'),
      keras.layers.Dense(1000, activation='relu'),
      keras.layers.Dense(10, activation='sigmoid'),]
  )
  model.compile(
      optimizer = 'SGD',
      loss = 'categorical_crossentropy',
      metrics = ['accuracy']
  )
  return model

In [23]:
train_tf_dataset = tf.data.Dataset.from_tensor_slices((X_train_scaled, y_train_categorical))
test_tf_dataset = tf.data.Dataset.from_tensor_slices((X_test_scaled, y_test_categorical))

In [24]:
strategy = tf.distribute.MirroredStrategy()

In [25]:
strategy.num_replicas_in_sync

1

In [26]:
BATCH_SIZE_PER_REPLICA = 250
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

train_dataset = train_tf_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_tf_dataset.batch(BATCH_SIZE)

In [27]:
%%timeit -n1 -r1
with strategy.scope():
  gpu_model = get_model()
  gpu_model.fit(train_dataset, epochs=50)

Epoch 1/50
200/200 [==============================] - 4s 11ms/step - loss: 1.9858 - accuracy: 0.2960
Epoch 2/50
200/200 [==============================] - 2s 11ms/step - loss: 1.8208 - accuracy: 0.3628
Epoch 3/50
200/200 [==============================] - 2s 11ms/step - loss: 1.7585 - accuracy: 0.3862
Epoch 4/50
200/200 [==============================] - 2s 11ms/step - loss: 1.7154 - accuracy: 0.4020
Epoch 5/50
200/200 [==============================] - 2s 10ms/step - loss: 1.6811 - accuracy: 0.4137
Epoch 6/50
200/200 [==============================] - 3s 15ms/step - loss: 1.6521 - accuracy: 0.4246
Epoch 7/50
200/200 [==============================] - 2s 11ms/step - loss: 1.6269 - accuracy: 0.4335
Epoch 8/50
200/200 [==============================] - 2s 10ms/step - loss: 1.6048 - accuracy: 0.4422
Epoch 9/50
200/200 [==============================] - 2s 10ms/step - loss: 1.5849 - accuracy: 0.4492
Epoch 10/50
200/200 [==============================] - 3s 13ms/step - loss: 1.5669 - accura